<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li></ul></li><li><span><a href="#Find-articles-to-be-coded" data-toc-modified-id="Find-articles-to-be-coded-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Find articles to be coded</a></span><ul class="toc-item"><li><span><a href="#which-articles-have-already-been-coded?" data-toc-modified-id="which-articles-have-already-been-coded?-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>which articles have already been coded?</a></span><ul class="toc-item"><li><span><a href="#Tag-the-coded-articles" data-toc-modified-id="Tag-the-coded-articles-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Tag the coded articles</a></span></li><li><span><a href="#Profile-the-coded-articles" data-toc-modified-id="Profile-the-coded-articles-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Profile the coded articles</a></span></li></ul></li><li><span><a href="#tag-all-local-news" data-toc-modified-id="tag-all-local-news-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>tag all local news</a></span><ul class="toc-item"><li><span><a href="#TODO" data-toc-modified-id="TODO-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>TODO</a></span></li><li><span><a href="#Grand-Rapids-Press-local-news" data-toc-modified-id="Grand-Rapids-Press-local-news-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Grand Rapids Press local news</a></span></li><li><span><a href="#Detroit-News-local-news" data-toc-modified-id="Detroit-News-local-news-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Detroit News local news</a></span></li></ul></li></ul></li><li><span><a href="#Code-Articles" data-toc-modified-id="Code-Articles-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Code Articles</a></span></li></ul></div>

# Introduction

- Back to [Table of Contents](#Table-of-Contents)

This is a notebook that expands on the OpenCalais code in the file `article_coding.py`, also in this folder.  It includes more sections on selecting publications you want to submit to OpenCalais as an example.  It is intended to be copied and re-used.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
debug_flag = True

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
import datetime
from django.db.models import Avg, Max, Min
import six

## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [ ]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [ ]:
%run $django_init_path

In [ ]:
# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.models import Article
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase

# Find articles to be coded

- Back to [Table of Contents](#Table-of-Contents)

Tag all locally implemented hard news articles in database and all that have already been coded using Open Calais V2, then work through using OpenCalais to code all local hard news that hasn't alredy been coded, starting with those proximal to the coding sample for methods paper.

## which articles have already been coded?

- Back to [Table of Contents](#Table-of-Contents)

More precisely, find all articles that have Article_Data coded by the automated coder with type "OpenCalais_REST_API_v2" and tag the articles as "coded-open_calais_v2" or something like that.

Then, for articles without that tag, use our criteria for local hard news to filter out and tag publications in the year before and after the month used to evaluate the automated coder, in both the Grand Rapids Press and the Detroit News, so I can look at longer time frames, then code all articles currently in database.

Eventually, then, we'll code and examine before and after layoffs.

In [ ]:
# look for publications that have article data:
# - coded by automated coder
# - with coder type of "OpenCalais_REST_API_v2"

# get automated coder
automated_coder_user = ArticleCoder.get_automated_coding_user()

print( "{} - Loaded automated user: {}, id = {}".format( datetime.datetime.now(), automated_coder_user, automated_coder_user.id ) )

In [ ]:
# try aggregates
article_qs = Article.objects.all()
pub_date_info = article_qs.aggregate( Max( 'pub_date' ), Min( 'pub_date' ) )
print( pub_date_info )

In [ ]:
# find articles with Article_Data created by the automated user...
article_qs = Article.objects.filter( article_data__coder = automated_coder_user )

# ...and specifically coded using OpenCalais V2...
article_qs = article_qs.filter( article_data__coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )

# ...and finally, we just want the distinct articles by ID.
article_qs = article_qs.order_by( "id" ).distinct( "id" )

# count?
article_count = article_qs.count()
print( "Found {} articles".format( article_count ) )

### Tag the coded articles

- Back to [Table of Contents](#Table-of-Contents)

Removing duplicates present from joining with Article_Data yields 579 articles that were coded by the automated coder.

Tag all the coded articles with `OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME`.

In [ ]:
# declare variables
current_article = None
tag_name_list = None
article_count = None
untagged_count = None
already_tagged_count = None
newly_tagged_count = None
count_sum = None
do_add_tag = False

# init
do_add_tag = True

# get article_count
article_count = article_qs.count()

# loop over articles.
untagged_count = 0
already_tagged_count = 0
newly_tagged_count = 0
for current_article in article_qs:
    
    # get list of tags for this publication
    tag_name_list = current_article.tags.names()
    
    # is the coded tag in the list?
    if ( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME not in tag_name_list ):
        
        # are we adding tag?
        if ( do_add_tag == True ):

            # add tag.
            current_article.tags.add( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )
            newly_tagged_count += 1
            
        else:

            # for now, increment untagged count
            untagged_count += 1
            
        #-- END check to see if we are adding tag. --#
        
    else:
        
        # already tagged
        already_tagged_count += 1
        
    #-- END check to see if coded tag is set --#
    
#-- END loop over articles. --#

print( "Article counts:" )
print( "- total articles: {}".format( article_count ) )
print( "- untagged articles: {}".format( untagged_count ) )
print( "- already tagged: {}".format( already_tagged_count ) )
print( "- newly tagged: {}".format( newly_tagged_count ) )
count_sum = untagged_count + already_tagged_count + newly_tagged_count
print( "- count sum: {}".format( count_sum ) )

### Profile the coded articles

- Back to [Table of Contents](#Table-of-Contents)

Look at range of pub dates.

In [ ]:
tags_in_list = []
tags_in_list.append( OpenCalaisV2ArticleCoder.TAG_CODED_BY_ME )
article_qs = Article.objects.filter( tags__name__in = tags_in_list )
print( "Matching article count: {}".format( article_qs.count() ) )

In [ ]:
# profile these publications
min_pub_date = None
max_pub_date = None
current_pub_date = None
pub_date_count = None
date_to_count_map = {}
date_to_articles_map = {}
pub_date_article_dict = None

# try aggregates
pub_date_info = article_qs.aggregate( Max( 'pub_date' ), Min( 'pub_date' ) )
print( pub_date_info )

# counts of pubs by date
for current_article in article_qs:
    
    # get pub_date
    current_pub_date = current_article.pub_date
    current_article_id = current_article.id
    
    # get count, increment, and store.
    pub_date_count = date_to_count_map.get( current_pub_date, 0 )
    pub_date_count += 1
    date_to_count_map[ current_pub_date ] = pub_date_count
    
    # also, store up ids and instances
    
    # get dict of article ids to article instances for date
    pub_date_article_dict = date_to_articles_map.get( current_pub_date, {} )
    
    # article already there?
    if ( current_article_id not in pub_date_article_dict ):
        
        # no - add it.
        pub_date_article_dict[ current_article_id ] = current_article
        
    #-- END check to see if article already there.
    
    # put dict back.
    date_to_articles_map[ current_pub_date ] = pub_date_article_dict
    
#-- END loop over articles. --#

# output dates and counts.

# get list of keys from map
keys_list = list( six.viewkeys( date_to_count_map ) )
keys_list.sort()
for current_pub_date in keys_list:
    
    # get count
    pub_date_count = date_to_count_map.get( current_pub_date, 0 )
    print( "- {} ( {} ) count: {}".format( current_pub_date, type( current_pub_date ), pub_date_count ) )
    
#-- END loop over dates --#

In [ ]:
# look at the 2010-07-31 date
pub_date = datetime.datetime.strptime( "2010-07-31", "%Y-%m-%d" ).date()
articles_for_date = date_to_articles_map.get( pub_date, {} )
print( articles_for_date )

# get the article and look at its tags.
article_instance = articles_for_date.get( 6065 )
print( article_instance.tags.all() )

# loop over associated Article_Data instances.
for article_data in article_instance.article_data_set.all():
    
    print( article_data )
    
#-- END loop over associated Article_Data instances --#

## tag all local news

- Back to [Table of Contents](#Table-of-Contents)

Definition of local hard news by in-house implementor for Grand Rapids Press and Detroit News follow.  For each, tag all articles in database that match as "local_hard_news".

### TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- make class for GRPB at NewsBank.

    - context_text/collectors/newsbank/newspapers/GRPB.py
    - abstract out shared stuff from GRPB.py and DTNB.py into abstract parent class context_text/collectors/newsbank/newspapers/newspaper.py
    - also, pull the things that are newspaper specific out of ArticleCoder.py and into the GRPB.py class.
    - update DTNB.py to use the parent class.
    - think how we specify which class to use for author strings - needs to be speced to an interface, but not just a newsbank one - so, abstraction here should be higher up - in shared?
    
- refine "local news" and "locally created" regular expressions for Grand Rapids Press based on contents of `author_string` and `author_affiliation`.
- do the same for TDN.
- then, use the updated classes and definitions below to flag all local hard news in database for each publication.

### Grand Rapids Press local news

- Back to [Table of Contents](#Table-of-Contents)

Grand Rapids Press local hard news:

- `context_text/examples/articles/articles-GRP-local_news.py`
- local hard news sections (stored in `Article.GRP_NEWS_SECTION_NAME_LIST`):

    - "Business"
    - "City and Region"
    - "Front Page"
    - "Lakeshore"
    - "Religion"
    - "Special"
    - "State"

- in-house implementor (based on byline patterns, stored in `sourcenet.models.Article.Q_GRP_IN_HOUSE_AUTHOR`):

    - Byline ends in "/ THE GRAND RAPIDS PRESS", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *THE GRAND RAPIDS PRESS$'`

    - Byline ends in "/ PRESS * EDITOR", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *PRESS .* EDITOR$' )`

    - Byline ends in "/ GRAND RAPIDS PRESS * BUREAU", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *GRAND RAPIDS PRESS .* BUREAU$' )`

    - Byline ends in "/ SPECIAL TO THE PRESS", ignore case.

        - `Q( author_varchar__iregex = r'.* */ *SPECIAL TO THE PRESS$' )`
        
- can also exclude columns (I will not):

        grp_article_qs = grp_article_qs.exclude( index_terms__icontains = "Column" )

Need to work to further refine this.

Looking at affiliation strings:

    SELECT author_affiliation, COUNT( author_affiliation ) as affiliation_count
    FROM context_text_article
    WHERE newspaper_id = 1
    GROUP BY author_affiliation
    ORDER BY COUNT( author_affiliation ) DESC;
    
And at author strings for collective bylines:

    SELECT author_string, COUNT( author_string ) as author_count
    FROM context_text_article
    WHERE newspaper_id = 1
    GROUP BY author_string
    ORDER BY COUNT( author_string ) DESC
    LIMIT 10;


In [ ]:
# filter queryset to just locally created Grand Rapids Press (GRP) articles.
# imports
from context_text.models import Article
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase
from context_text.collectors.newsbank.newspapers.GRPB import GRPB

# declare variables - Grand Rapids Press
do_apply_tag = False
tag_to_apply = None
grp_local_news_sections = []
grp_newspaper = None
grp_article_qs = None



current_article = None
author_string = ""
slash_index = -1
affiliation = ""
article_count = -1
article_counter = -1
article_id_list = []
tags_in_list = []
tags_not_in_list = []
filter_out_prelim_tags = False
random_count = -1
article_counter = -1

# declare variables - size of random sample we want
#random_count = 60

# declare variables - also, apply tag?
do_apply_tag = False
tag_to_apply = ContextTextBase.TAG_LOCAL_HARD_NEWS

# set up "local, regional and state news" sections
grp_local_news_sections = Article.GRP_NEWS_SECTION_NAME_LIST

# Grand Rapids Press
# get newspaper instance for GRP.
grp_newspaper = Newspaper.objects.get( id = 1 )

'''
# populate author_affiliation in Article table for GR Press.
grp_article_qs = Article.objects.filter( newspaper = grp_newspaper )
grp_article_qs = grp_article_qs.filter( author_string__contains = "/" )
grp_article_qs = grp_article_qs.filter( author_affiliation__isnull = True )

article_count = grp_article_qs.count()
article_counter = 0
for current_article in grp_article_qs:

    article_counter = article_counter + 1

    # output article.
    print( "- Article " + str( article_counter ) + " of " + str( article_count ) + ": " + str( current_article ) )
    
    # get author_string
    author_string = current_article.author_string
    
    # find "/"
    slash_index = author_string.find( "/" )
    
    # got one?
    if ( slash_index > -1 ):
    
        # yes.  get everything after the slash.
        affiliation = author_string[ ( slash_index + 1 ) : ]
        
        # strip off white space
        affiliation = affiliation.strip()
        
        print( "    - Affiliation = \"" + affiliation + "\"" )
        
        current_article.author_affiliation = affiliation
        current_article.save()
        
    #-- END check to see if "/" present.
    
#-- END loop over articles. --#
'''

# start with all articles
grp_article_qs = Article.objects.all()

'''
# date range, newspaper, section list, and in-house reporters.
grp_article_qs = Article.filter_articles( qs_IN = grp_article_qs,
                                          start_date = "2009-12-01",
                                          end_date = "2009-12-31",
                                          newspaper = grp_newspaper,
                                          section_name_list = grp_local_news_sections,
                                          custom_article_q = Article.Q_GRP_IN_HOUSE_AUTHOR )
'''

# now, need to find local news articles to test on.
#grp_article_qs = grp_article_qs.filter( newspaper = grp_newspaper )

# only the locally implemented sections
#grp_article_qs = grp_article_qs.filter( section__in = grp_local_news_sections )

# and, with an in-house author
#grp_article_qs = grp_article_qs.filter( Article.Q_GRP_IN_HOUSE_AUTHOR )

# and no columns
grp_article_qs = grp_article_qs.exclude( index_terms__icontains = "Column" )

# how many is that?
article_count = grp_article_qs.count()

print( "Article count before filtering on tags: " + str( article_count ) )

# ==> tags

# tags to exclude
#tags_not_in_list = [ "prelim_reliability", "prelim_network" ]
tags_not_in_list = [ "minnesota1-20160328", "minnesota2-20160328", ]
if ( len( tags_not_in_list ) > 0 ):

    # exclude those in a list
    print( "filtering out articles with tags: " + str( tags_not_in_list ) )
    grp_article_qs = grp_article_qs.exclude( tags__name__in = tags_not_in_list )

#-- END check to see if we have a specific list of tags we want to exclude --#

# include only those with certain tags.
#tags_in_list = [ "prelim_unit_test_001", "prelim_unit_test_002", "prelim_unit_test_003", "prelim_unit_test_004", "prelim_unit_test_005", "prelim_unit_test_006", "prelim_unit_test_007" ]
tags_in_list = [ "grp_month", ]
if ( len( tags_in_list ) > 0 ):

    # filter
    print( "filtering to just articles with tags: " + str( tags_in_list ) )
    grp_article_qs = grp_article_qs.filter( tags__name__in = tags_in_list )
    
#-- END check to see if we have a specific list of tags we want to include --#

# filter out "*prelim*" tags?
#filter_out_prelim_tags = True
if ( filter_out_prelim_tags == True ):

    # ifilter out all articles with any tag whose name contains "prelim".
    print( "filtering out articles with tags that contain \"prelim\"" )
    grp_article_qs = grp_article_qs.exclude( tags__name__icontains = "prelim" )
    
#-- END check to see if we filter out "prelim_*" tags --#

# how many is that?
article_count = grp_article_qs.count()

print( "Article count after tag filtering: " + str( article_count ) )

# do we want a random sample?
random_count = 147
if ( random_count > 0 ):

    # to get random, order them by "?", then use slicing to retrieve requested
    #     number.
    grp_article_qs = grp_article_qs.order_by( "?" )[ : random_count ]
    
#-- END check to see if we want random sample --#

# this is a nice algorithm, also:
# - http://www.titov.net/2005/09/21/do-not-use-order-by-rand-or-how-to-get-random-rows-from-table/

# make list of article IDs.
article_id_list = []
article_counter = 0
for current_article in grp_article_qs:

    # increment article_counter
    article_counter += 1

    # add IDs to article_id_list
    article_id_list.append( str( current_article.id ) )
    
    # apply a tag while we are at it?
    if ( ( do_apply_tag == True ) and ( tag_to_apply is not None ) and ( tag_to_apply != "" ) ):
    
        # yes, please.  Add tag.
        current_article.tags.add( tag_to_apply )
        
    #-- END check to see if we apply tag. --#

    # output the tags.
    print( "- Tags for article " + str( current_article.id ) + " : " + str( current_article.tags.all() ) )
   
#-- END loop over articles --#

# output the list.
print( "Found " + str( article_counter ) + " articles ( " + str( article_count ) + " )." )
print( "List of " + str( len( article_id_list ) ) + " local GRP staff article IDs: " + ", ".join( article_id_list ) )

### Detroit News local news

- Back to [Table of Contents](#Table-of-Contents)

Detroit News local news:

- `context_text/examples/articles/articles-TDN-local_news.py`
- local hard news sections (stored in `from context_text.collectors.newsbank.newspapers.DTNB import DTNB` - `DTNB.NEWS_SECTION_NAME_LIST`):

    - "Business"
    - "Metro"
    - "Nation" - because of auto industry stories

- in-house implementor (based on byline patterns, stored in `DTNB.Q_IN_HOUSE_AUTHOR`):

    - Byline ends in "/ The Detroit News", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*the\s*detroit\s*news$' )`

    - Byline ends in "Special to The Detroit News", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*special\s*to\s*the\s*detroit\s*news$' )`

    - Byline ends in "Detroit News * Bureau", ignore case.

        - `Q( author_varchar__iregex = r'.*\s*/\s*detroit\s*news\s*.*\s*bureau$' )`   

# Code Articles

- Back to [Table of Contents](#Table-of-Contente)

In [ ]:
# declare variables

# declare variables - article filter parameters
start_pub_date = None # should be datetime instance
end_pub_date = None # should be datetime instance
tag_in_list = []
paper_id_in_list = []
section_list = []
article_id_in_list = []
params = {}

# declare variables - processing
do_i_print_updates = True
my_article_coding = None
article_qs = None
article_count = -1
coding_status = ""
limit_to = -1
do_coding = False

# declare variables - results
success_count = -1
success_list = None
got_errors = False
error_count = -1
error_dictionary = None
error_article_id = -1
error_status_list = None
error_status = ""
error_status_counter = -1

# first, get a list of articles to code.

# ! Set param values.

# ==> start and end dates
#start_pub_date = "2009-12-06"
#end_pub_date = "2009-12-12"

# ==> tagged articles
#tag_in_list = "prelim_reliability"
#tag_in_list = "prelim_network"
#tag_in_list = "prelim_unit_test_007"
#tag_in_list = [ "prelim_reliability", "prelim_network" ]
#tag_in_list = [ "prelim_reliability_test" ] # 60 articles - Grand Rapids only.
#tag_in_list = [ "prelim_reliability_combined" ] # 87 articles, Grand Rapids and Detroit.
#tag_in_list = [ "prelim_training_001" ]
tag_in_list = [ "grp_month" ]

# ==> IDs of newspapers to include.
#paper_id_in_list = "1"

# ==> names of sections to include.
#section_list = "Lakeshore,Front Page,City and Region,Business"

# ==> just limit to specific articles by ID.
#article_id_in_list = [ 360962 ]
#article_id_in_list = [ 28598 ]
#article_id_in_list = [ 21653, 21756 ]
#article_id_in_list = [ 90948 ]
#article_id_in_list = [ 21627, 21609, 21579 ]
#article_id_in_list = [ 48778 ]
#article_id_in_list = [ 6065 ]
#article_id_in_list = [ 221858 ]
#article_id_in_list = [ 23804, 22630 ]
article_id_in_list = [ 23804 ]

# filter parameters
params[ ArticleCoding.PARAM_START_DATE ] = start_pub_date
params[ ArticleCoding.PARAM_END_DATE ] = end_pub_date
params[ ArticleCoding.PARAM_TAG_LIST ] = tag_in_list
params[ ArticleCoding.PARAM_PUBLICATION_LIST ] = paper_id_in_list
params[ ArticleCoding.PARAM_SECTION_LIST ] = section_list
params[ ArticleCoding.PARAM_ARTICLE_ID_LIST ] = article_id_in_list

# set coder you want to use.

# OpenCalais REST API v.2
params[ ArticleCoding.PARAM_CODER_TYPE ] = ArticleCoding.ARTICLE_CODING_IMPL_OPEN_CALAIS_API_V2

# get instance of ArticleCoding
my_article_coding = ArticleCoding()
my_article_coding.do_print_updates = do_i_print_updates

# set params
my_article_coding.store_parameters( params )

# create query set - ArticleCoding does the filtering for you.
article_qs = my_article_coding.create_article_query_set()

# limit for an initial test?
if ( ( limit_to is not None ) and ( isinstance( limit_to, int ) == True ) and ( limit_to > 0 ) ):

    # yes.
    article_qs = article_qs[ : limit_to ]

#-- END check to see if limit --#

# get article count
article_count = article_qs.count()

print( "Query params:" )
print( params )
print( "Matching article count: " + str( article_count ) )

# Do coding?
if ( do_coding == True ):

    print( "do_coding == True - it's on!" )

    # yes - make sure we have at least one article:
    if ( article_count > 0 ):

        # invoke the code_article_data( self, query_set_IN ) method.
        coding_status = my_article_coding.code_article_data( article_qs )
    
        # output status
        print( "\n\n==============================\n\nCoding status: \"" + coding_status + "\"" )
        
        # get success count
        success_count = my_article_coding.get_success_count()
        print( "\n\n====> Count of articles successfully processed: " + str( success_count ) )    
        
        # if successes, list out IDs.
        if ( success_count > 0 ):
        
            # there were successes.
            success_list = my_article_coding.get_success_list()
            print( "- list of successfully processed articles: " + str( success_list ) )
        
        #-- END check to see if successes. --#
        
        # got errors?
        got_errors = my_article_coding.has_errors()
        if ( got_errors == True ):
        
            # get error dictionary
            error_dictionary = my_article_coding.get_error_dictionary()
            
            # get error count
            error_count = len( error_dictionary )
            print( "\n\n====> Count of articles with errors: " + str( error_count ) )
            
            # loop...
            for error_article_id, error_status_list in six.iteritems( error_dictionary ):
            
                # output errors for this article.
                print( "- errors for article ID " + str( error_article_id ) + ":" )
                
                # loop over status messages.
                error_status_counter = 0
                for error_status in error_status_list:
                
                    # increment status
                    error_status_counter += 1

                    # print status
                    print( "----> status #" + str( error_status_counter ) + ": " + error_status )
                    
                #-- END loop over status messages. --#
            
            #-- END loop over articles. --#
   
        #-- END check to see if errors --#
    
    #-- END check to see if article count. --#
    
else:
    
    # output matching article count.
    print( "do_coding == False, so dry run" )
    
#-- END check to see if we do_coding --#
